# Ajuste de Tipos de Datos en PostgreSQL

Este notebook tiene como objetivo verificar y ajustar los tipos de datos de las columnas en PostgreSQL después de la carga inicial desde archivos CSV.

## Objetivo

Cuando se cargan datos desde CSV usando `COPY` sin especificar tipos de datos explícitos en pandas, PostgreSQL puede inferir tipos incorrectos o almacenar datos como `VARCHAR`/`TEXT` cuando deberían ser tipos específicos como `NUMERIC`, `TIMESTAMP` o `INTEGER`.

Este notebook:
1. Carga cada tabla en un DataFrame de pandas
2. Verifica los tipos de datos actuales usando `.info()`
3. Identifica discrepancias entre los tipos actuales y los esperados
4. Ejecuta los ajustes necesarios usando SQL `ALTER TABLE` cuando sea requerido


## Tabla de Verificación de Tipos de Datos

A continuación se presenta la lista completa de las **16 columnas** que requieren verificación y ajuste de tipos de datos, agrupadas por tabla:


| Tabla | Columna | Tipo de Dato Esperado | Descripción |
|-------|---------|----------------------|-------------|
| **usuarios** | `fecha_registro` | TIMESTAMP | Fecha de registro del usuario |
| **productos** | `precio` | NUMERIC(10, 2) | Precio unitario del producto |
| **productos** | `stock` | INTEGER | Cantidad disponible en inventario |
| **ordenes** | `fecha_orden` | TIMESTAMP | Fecha y hora de creación de la orden |
| **ordenes** | `total` | NUMERIC(10, 2) | Monto total de la orden |
| **ordenes** | `estado` | ENUM | Estado de la orden (Pendiente, Enviado, Completado, Cancelado) |
| **detalle_ordenes** | `precio_unitario` | NUMERIC(10, 2) | Precio unitario al momento de la orden |
| **detalle_ordenes** | `cantidad` | INTEGER | Cantidad de productos en el detalle |
| **carrito** | `fecha_agregado` | TIMESTAMP | Fecha en que se agregó el producto al carrito |
| **carrito** | `cantidad` | INTEGER | Cantidad del producto en el carrito |
| **ordenes_metodos_pago** | `monto_pagado` | NUMERIC(10, 2) | Monto pagado con cada método |
| **resenas_productos** | `fecha` | TIMESTAMP | Fecha de publicación de la reseña |
| **resenas_productos** | `calificacion` | INTEGER | Calificación del producto (1-5) |
| **historial_pagos** | `monto` | NUMERIC(10, 2) | Monto del pago |
| **historial_pagos** | `fecha_pago` | TIMESTAMP | Fecha del pago |
| **historial_pagos** | `estado_pago` | ENUM | Estado del pago (Procesando, Pagado, Fallido, Reembolsado) |

**Total: 16 columnas requieren verificación de tipos de datos**


## 1. Importaciones y Configuración Inicial


In [1]:
# Importar bibliotecas necesarias
import sys
import os

try:
    from ..Utils.path_manager import PathManager
except ImportError:
    # Si falla el import relativo 
    current_dir = os.getcwd()  # Directorio actual de trabajo
    if 'Notebooks' in current_dir:
        avance1_dir = os.path.dirname(current_dir)
    else:
        # Intentar calcular desde la ubicación relativa
        avance1_dir = os.path.abspath(os.path.join(current_dir, '..'))
    
    utils_dir = os.path.join(avance1_dir, 'Utils')
    if utils_dir not in sys.path:
        sys.path.insert(0, utils_dir)
    from path_manager import PathManager

# Configurar sys.path usando PathManager (patrón Singleton)
path_manager = PathManager.get_instance()
path_manager.setup_sys_path()

# Importar DBConnector desde la raíz del proyecto
from Database.db_connector import DBConnector

# Importar bibliotecas de análisis
import pandas as pd
import numpy as np

print("✓ Bibliotecas importadas correctamente")


✓ Bibliotecas importadas correctamente


In [2]:
# Obtener el Engine de la base de datos usando DBConnector (patrón Singleton)
db = DBConnector.get_instance()
engine = db.get_engine()

print("✓ Conexión a la base de datos establecida")
print(f"✓ Engine obtenido: {type(engine).__name__}")


✓ Conexión a la base de datos establecida
✓ Engine obtenido: Engine


### Función para Cargar Tablas

Función genérica para cargar cualquier tabla de la base de datos en un DataFrame de pandas.


In [3]:
def load_table_to_df(table_name, engine):
    """
    Carga una tabla completa de la base de datos PostgreSQL en un DataFrame de pandas.
    
    Args:
        table_name (str): Nombre de la tabla a cargar
        engine: Engine de SQLAlchemy para la conexión a la base de datos
    
    Returns:
        pd.DataFrame: DataFrame con todos los datos de la tabla
    
    Example:
        >>> df_usuarios = load_table_to_df('usuarios', engine)
    """
    # Una consulta sencilla para cargar la tabla completa
    query = f"SELECT * FROM {table_name};"
    
    return pd.read_sql(query, engine)


## 2. Verificación de Tipos de Datos por Tabla

A continuación cargamos cada tabla y verificamos los tipos de datos actuales usando `.info()` y comparamos con los tipos esperados.


### 2.1. Tabla: usuarios

**Columnas a verificar:**
- `fecha_registro` → TIMESTAMP


In [4]:
# Cargar tabla usuarios
df_usuarios = load_table_to_df('usuarios', engine)

print("=" * 80)
print("TABLA: usuarios")
print("=" * 80)
print(f"\nForma del DataFrame: {df_usuarios.shape}")
print(f"\nPrimeras filas:")
print(df_usuarios.head())
print(f"\nInformación de tipos de datos:")
df_usuarios.info()
print(f"\nTipos de datos por columna:")
print(df_usuarios.dtypes)


TABLA: usuarios

Forma del DataFrame: (1000, 7)

Primeras filas:
   usuario_id        nombre   apellido       dni  \
0           1       Encarna    Donaire  49877134   
1           2  Jose Ignacio    Canales  98778810   
2           3          José  Sebastián  80095714   
3           4      Priscila      Vilar  94128750   
4           5      Epifanio       Ríos  62958342   

                              email     contraseña             fecha_registro  
0       encarna.donaire1@correo.com  Contraseña123 2025-11-09 05:17:54.903678  
1  jose ignacio.canales2@correo.com  Contraseña123 2025-11-09 05:17:54.903678  
2        josé.sebastián3@correo.com  Contraseña123 2025-11-09 05:17:54.903678  
3        priscila.vilar4@correo.com  Contraseña123 2025-11-09 05:17:54.903678  
4         epifanio.ríos5@correo.com  Contraseña123 2025-11-09 05:17:54.903678  

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   

### 2.2. Tabla: productos

**Columnas a verificar:**
- `precio` → NUMERIC(10, 2)
- `stock` → INTEGER


In [5]:
# Cargar tabla productos
df_productos = load_table_to_df('productos', engine)

print("=" * 80)
print("TABLA: productos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_productos.shape}")
print(f"\nPrimeras filas:")
print(df_productos.head())
print(f"\nInformación de tipos de datos:")
df_productos.info()
print(f"\nTipos de datos por columna:")
print(df_productos.dtypes)


TABLA: productos

Forma del DataFrame: (36, 6)

Primeras filas:
   producto_id                      nombre  \
0            1       Smartphone Galaxy A54   
1            2     Laptop Dell Inspiron 15   
2            3  Auriculares Bluetooth Sony   
3            4      Camiseta Básica Hombre   
4            5          Jeans Skinny Mujer   

                                         descripcion  precio  stock  \
0  Teléfono inteligente con pantalla AMOLED y cám...  349.99     50   
1  Laptop para trabajo y estudio con procesador I...  799.00     30   
2  Auriculares inalámbricos con cancelación de ru...  129.99    100   
3  Camiseta de algodón 100% disponible en varias ...   14.99    200   
4              Jeans corte skinny de mezclilla azul.   39.99    150   

   categoria_id  
0             1  
1             1  
2             1  
3             2  
4             2  

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 6 

### 2.3. Tabla: ordenes

**Columnas a verificar:**
- `fecha_orden` → TIMESTAMP
- `total` → NUMERIC(10, 2)
- `estado` → ENUM


In [6]:
# Cargar tabla ordenes
df_ordenes = load_table_to_df('ordenes', engine)

print("=" * 80)
print("TABLA: ordenes")
print("=" * 80)
print(f"\nForma del DataFrame: {df_ordenes.shape}")
print(f"\nPrimeras filas:")
print(df_ordenes.head())
print(f"\nInformación de tipos de datos:")
df_ordenes.info()
print(f"\nTipos de datos por columna:")
print(df_ordenes.dtypes)


TABLA: ordenes

Forma del DataFrame: (10000, 5)

Primeras filas:
   orden_id  usuario_id         fecha_orden   total      estado
0         1         153 2024-09-09 09:10:27  393.81     Enviado
1         2         810 2024-09-29 09:10:27  918.85  Completado
2         3         404 2024-11-03 09:10:27  391.29   Pendiente
3         4          52 2024-09-14 09:10:27  250.03   Cancelado
4         5         811 2025-04-11 09:10:27  983.13  Completado

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   orden_id     10000 non-null  int64         
 1   usuario_id   10000 non-null  int64         
 2   fecha_orden  10000 non-null  datetime64[ns]
 3   total        10000 non-null  float64       
 4   estado       10000 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usa

### 2.4. Tabla: detalle_ordenes

**Columnas a verificar:**
- `precio_unitario` → NUMERIC(10, 2)
- `cantidad` → INTEGER


In [7]:
# Cargar tabla detalle_ordenes
df_detalle_ordenes = load_table_to_df('detalle_ordenes', engine)

print("=" * 80)
print("TABLA: detalle_ordenes")
print("=" * 80)
print(f"\nForma del DataFrame: {df_detalle_ordenes.shape}")
print(f"\nPrimeras filas:")
print(df_detalle_ordenes.head())
print(f"\nInformación de tipos de datos:")
df_detalle_ordenes.info()
print(f"\nTipos de datos por columna:")
print(df_detalle_ordenes.dtypes)


TABLA: detalle_ordenes

Forma del DataFrame: (10000, 5)

Primeras filas:
   detalle_id  orden_id  producto_id  cantidad  precio_unitario
0           1       833           33         1           425.26
1           2       657           21         4           307.13
2           3       911           15         4           444.75
3           4       818           26         5           346.24
4           5       695           12         3           155.05

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   detalle_id       10000 non-null  int64  
 1   orden_id         10000 non-null  int64  
 2   producto_id      10000 non-null  int64  
 3   cantidad         10000 non-null  int64  
 4   precio_unitario  10000 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 390.8 KB

Tipos de datos por columna:
d

### 2.5. Tabla: carrito

**Columnas a verificar:**
- `fecha_agregado` → TIMESTAMP
- `cantidad` → INTEGER


In [8]:
# Cargar tabla carrito
df_carrito = load_table_to_df('carrito', engine)

print("=" * 80)
print("TABLA: carrito")
print("=" * 80)
print(f"\nForma del DataFrame: {df_carrito.shape}")
print(f"\nPrimeras filas:")
print(df_carrito.head())
print(f"\nInformación de tipos de datos:")
df_carrito.info()
print(f"\nTipos de datos por columna:")
print(df_carrito.dtypes)


TABLA: carrito

Forma del DataFrame: (5000, 5)

Primeras filas:
   carrito_id  usuario_id  producto_id  cantidad      fecha_agregado
0           1         844           22         3 2025-04-29 10:15:10
1           2         901            7         2 2025-05-14 10:15:10
2           3         824           28         1 2025-04-26 10:15:10
3           4          94            5         3 2025-04-20 10:15:10
4           5         228           15         3 2025-05-11 10:15:10

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   carrito_id      5000 non-null   int64         
 1   usuario_id      5000 non-null   int64         
 2   producto_id     5000 non-null   int64         
 3   cantidad        5000 non-null   int64         
 4   fecha_agregado  5000 non-null   datetime64[ns]
dtypes: datetime64[n

### 2.6. Tabla: ordenes_metodos_pago

**Columnas a verificar:**
- `monto_pagado` → NUMERIC(10, 2)


In [9]:
# Cargar tabla ordenes_metodos_pago
df_ordenes_metodos_pago = load_table_to_df('ordenes_metodos_pago', engine)

print("=" * 80)
print("TABLA: ordenes_metodos_pago")
print("=" * 80)
print(f"\nForma del DataFrame: {df_ordenes_metodos_pago.shape}")
print(f"\nPrimeras filas:")
print(df_ordenes_metodos_pago.head())
print(f"\nInformación de tipos de datos:")
df_ordenes_metodos_pago.info()
print(f"\nTipos de datos por columna:")
print(df_ordenes_metodos_pago.dtypes)


TABLA: ordenes_metodos_pago

Forma del DataFrame: (10000, 4)

Primeras filas:
   orden_metodo_id  orden_id  metodo_pago_id  monto_pagado
0                1         1               2        580.22
1                2         2               4        638.87
2                3         3               7        946.09
3                4         4               2        390.35
4                5         5               3        908.67

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   orden_metodo_id  10000 non-null  int64  
 1   orden_id         10000 non-null  int64  
 2   metodo_pago_id   10000 non-null  int64  
 3   monto_pagado     10000 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 312.6 KB

Tipos de datos por columna:
orden_metodo_id      int64
orden_id             int64
metodo_pago_id    

### 2.7. Tabla: resenas_productos

**Columnas a verificar:**
- `fecha` → TIMESTAMP
- `calificacion` → INTEGER


In [10]:
# Cargar tabla resenas_productos
df_resenas_productos = load_table_to_df('resenas_productos', engine)

print("=" * 80)
print("TABLA: resenas_productos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_resenas_productos.shape}")
print(f"\nPrimeras filas:")
print(df_resenas_productos.head())
print(f"\nInformación de tipos de datos:")
df_resenas_productos.info()
print(f"\nTipos de datos por columna:")
print(df_resenas_productos.dtypes)


TABLA: resenas_productos

Forma del DataFrame: (7172, 6)

Primeras filas:
   resena_id  usuario_id  producto_id  calificacion  \
0          1         700           16             3   
1          2         557           20             2   
2          3         448           33             2   
3          4         483           11             5   
4          5         660           15             5   

                                          comentario               fecha  
0  El producto es aceptable por el precio. Quisqu... 2024-12-20 10:22:06  
1  No cumplió con lo prometido. Tempore distincti... 2024-12-26 10:22:06  
2  No cumplió con lo prometido. Modi doloribus re... 2024-07-21 10:22:06  
3  ¡Recomendado al 100%! Soluta consequatur paria... 2025-05-15 10:22:06  
4  ¡Recomendado al 100%! Nulla unde ipsum similiq... 2024-07-05 10:22:06  

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7172 entries, 0 to 7171
Data columns (total 6 columns):
 #   Co

### 2.8. Tabla: historial_pagos

**Columnas a verificar:**
- `monto` → NUMERIC(10, 2)
- `fecha_pago` → TIMESTAMP
- `estado_pago` → ENUM


In [11]:
# Cargar tabla historial_pagos
df_historial_pagos = load_table_to_df('historial_pagos', engine)

print("=" * 80)
print("TABLA: historial_pagos")
print("=" * 80)
print(f"\nForma del DataFrame: {df_historial_pagos.shape}")
print(f"\nPrimeras filas:")
print(df_historial_pagos.head())
print(f"\nInformación de tipos de datos:")
df_historial_pagos.info()
print(f"\nTipos de datos por columna:")
print(df_historial_pagos.dtypes)


TABLA: historial_pagos

Forma del DataFrame: (10000, 6)

Primeras filas:
   pago_id  orden_id  metodo_pago_id   monto          fecha_pago  estado_pago
0        1         1               4  147.13 2025-06-02 10:19:16   Procesando
1        2         2               1  647.98 2025-04-27 10:19:16  Reembolsado
2        3         3               6  627.45 2025-05-07 10:19:16      Fallido
3        4         4               5  848.07 2025-06-30 10:19:16   Procesando
4        5         5               5  189.80 2025-04-03 10:19:16      Fallido

Información de tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   pago_id         10000 non-null  int64         
 1   orden_id        10000 non-null  int64         
 2   metodo_pago_id  10000 non-null  int64         
 3   monto           10000 non-null  float64       
 4   fec

## 3. Verificación de Tipos de Datos en PostgreSQL

A continuación verificamos los tipos de datos actuales directamente desde PostgreSQL usando `information_schema`:


In [12]:
# Consulta para verificar tipos de datos actuales en PostgreSQL
query_tipos = """
SELECT 
    table_name, 
    column_name, 
    data_type, 
    numeric_precision, 
    numeric_scale,
    udt_name
FROM information_schema.columns
WHERE table_schema = 'public'
    AND column_name IN (
        'fecha_registro', 'precio', 'stock', 'fecha_orden', 'total', 'estado',
        'precio_unitario', 'cantidad', 'fecha_agregado', 'monto_pagado',
        'fecha', 'calificacion', 'monto', 'fecha_pago', 'estado_pago'
    )
ORDER BY table_name, column_name;
"""

df_tipos_actuales = pd.read_sql(query_tipos, engine)

print("=" * 80)
print("TIPOS DE DATOS ACTUALES EN POSTGRESQL")
print("=" * 80)
print(df_tipos_actuales.to_string(index=False))


TIPOS DE DATOS ACTUALES EN POSTGRESQL
          table_name     column_name                   data_type  numeric_precision  numeric_scale     udt_name
             carrito        cantidad                     integer               32.0            0.0         int4
             carrito  fecha_agregado timestamp without time zone                NaN            NaN    timestamp
     detalle_ordenes        cantidad                     integer               32.0            0.0         int4
     detalle_ordenes precio_unitario                     numeric               10.0            2.0      numeric
   direcciones_envio          estado           character varying                NaN            NaN      varchar
     historial_pagos     estado_pago                USER-DEFINED                NaN            NaN  estado_pago
     historial_pagos      fecha_pago timestamp without time zone                NaN            NaN    timestamp
     historial_pagos           monto                     numeric  

## 4. Comparación de Tipos Esperados vs Actuales

A continuación creamos una tabla comparativa para identificar discrepancias:


In [13]:
# Definir tipos esperados
tipos_esperados = {
    ('usuarios', 'fecha_registro'): 'TIMESTAMP',
    ('productos', 'precio'): 'NUMERIC(10, 2)',
    ('productos', 'stock'): 'INTEGER',
    ('ordenes', 'fecha_orden'): 'TIMESTAMP',
    ('ordenes', 'total'): 'NUMERIC(10, 2)',
    ('ordenes', 'estado'): 'ENUM',
    ('detalle_ordenes', 'precio_unitario'): 'NUMERIC(10, 2)',
    ('detalle_ordenes', 'cantidad'): 'INTEGER',
    ('carrito', 'fecha_agregado'): 'TIMESTAMP',
    ('carrito', 'cantidad'): 'INTEGER',
    ('ordenes_metodos_pago', 'monto_pagado'): 'NUMERIC(10, 2)',
    ('resenas_productos', 'fecha'): 'TIMESTAMP',
    ('resenas_productos', 'calificacion'): 'INTEGER',
    ('historial_pagos', 'monto'): 'NUMERIC(10, 2)',
    ('historial_pagos', 'fecha_pago'): 'TIMESTAMP',
    ('historial_pagos', 'estado_pago'): 'ENUM'
}

# Crear DataFrame comparativo
comparacion = []
for (table, column), tipo_esperado in tipos_esperados.items():
    # Buscar el tipo actual en df_tipos_actuales
    tipo_actual = df_tipos_actuales[
        (df_tipos_actuales['table_name'] == table) & 
        (df_tipos_actuales['column_name'] == column)
    ]
    
    if not tipo_actual.empty:
        data_type = tipo_actual.iloc[0]['data_type']
        udt_name = tipo_actual.iloc[0]['udt_name']
        numeric_precision = tipo_actual.iloc[0]['numeric_precision']
        numeric_scale = tipo_actual.iloc[0]['numeric_scale']
        
        # Formatear tipo actual para comparación
        if data_type == 'numeric':
            tipo_actual_str = f"NUMERIC({numeric_precision}, {numeric_scale})"
        elif data_type == 'timestamp without time zone':
            tipo_actual_str = 'TIMESTAMP'
        elif data_type == 'integer':
            tipo_actual_str = 'INTEGER'
        elif data_type == 'USER-DEFINED' and 'enum' in udt_name.lower():
            tipo_actual_str = 'ENUM'
        else:
            tipo_actual_str = f"{data_type} ({udt_name})"
        
        necesita_ajuste = tipo_actual_str != tipo_esperado
        
        comparacion.append({
            'Tabla': table,
            'Columna': column,
            'Tipo Esperado': tipo_esperado,
            'Tipo Actual': tipo_actual_str,
            'Necesita Ajuste': 'SÍ' if necesita_ajuste else 'NO'
        })
    else:
        comparacion.append({
            'Tabla': table,
            'Columna': column,
            'Tipo Esperado': tipo_esperado,
            'Tipo Actual': 'NO ENCONTRADO',
            'Necesita Ajuste': 'SÍ'
        })

df_comparacion = pd.DataFrame(comparacion)

print("=" * 80)
print("COMPARACIÓN: TIPOS ESPERADOS vs ACTUALES")
print("=" * 80)
print(df_comparacion.to_string(index=False))

print("\n" + "=" * 80)
print("RESUMEN DE AJUSTES NECESARIOS")
print("=" * 80)
ajustes_necesarios = df_comparacion[df_comparacion['Necesita Ajuste'] == 'SÍ']
print(f"\nTotal de columnas que requieren ajuste: {len(ajustes_necesarios)}")
if len(ajustes_necesarios) > 0:
    print("\nColumnas que requieren ajuste:")
    print(ajustes_necesarios[['Tabla', 'Columna', 'Tipo Esperado', 'Tipo Actual']].to_string(index=False))


COMPARACIÓN: TIPOS ESPERADOS vs ACTUALES
               Tabla         Columna  Tipo Esperado                 Tipo Actual Necesita Ajuste
            usuarios  fecha_registro      TIMESTAMP                   TIMESTAMP              NO
           productos          precio NUMERIC(10, 2)          NUMERIC(10.0, 2.0)              SÍ
           productos           stock        INTEGER                     INTEGER              NO
             ordenes     fecha_orden      TIMESTAMP                   TIMESTAMP              NO
             ordenes           total NUMERIC(10, 2)          NUMERIC(10.0, 2.0)              SÍ
             ordenes          estado           ENUM USER-DEFINED (estado_orden)              SÍ
     detalle_ordenes precio_unitario NUMERIC(10, 2)          NUMERIC(10.0, 2.0)              SÍ
     detalle_ordenes        cantidad        INTEGER                     INTEGER              NO
             carrito  fecha_agregado      TIMESTAMP                   TIMESTAMP              NO

## 5. CONCLUSIONES

### 5.1. Ajuste de Columnas NUMERIC(10, 2)

**ANÁLISIS DE RESULTADOS**: 

Durante la verificación se encontró una diferencia visual entre el tipo esperado `NUMERIC(10, 2)` y el tipo actual `NUMERIC(10.0, 2.0)` en las siguientes columnas:
- `productos.precio`
- `ordenes.total`
- `detalle_ordenes.precio_unitario`
- `ordenes_metodos_pago.monto_pagado`
- `historial_pagos.monto`

**CONCLUSIÓN**: Aunque existe esta diferencia en la representación (con decimales vs sin decimales), **NO es necesario realizar ningún cambio**. En PostgreSQL, `NUMERIC(10, 2)` y `NUMERIC(10.0, 2.0)` son funcionalmente idénticos: ambos definen un tipo numérico con 10 dígitos totales y 2 decimales. La diferencia es solo cosmética en cómo se muestra en `information_schema`, donde los valores de precisión y escala se almacenan como números de punto flotante.

Los tipos de datos actuales son correctos y adecuados para almacenar valores monetarios con precisión exacta.


### 5.2. Ajuste de Columnas TIMESTAMP

**ANÁLISIS DE RESULTADOS**: 

Durante la verificación se analizaron las siguientes columnas de tipo TIMESTAMP:
- `usuarios.fecha_registro`
- `ordenes.fecha_orden`
- `carrito.fecha_agregado`
- `resenas_productos.fecha`
- `historial_pagos.fecha_pago`

**CONCLUSIÓN**: **No se encontraron tipos por cambiar**. Todas las columnas de tipo TIMESTAMP ya están correctamente configuradas en PostgreSQL como `timestamp without time zone`, que es equivalente al tipo `TIMESTAMP` esperado. Los datos se están almacenando y recuperando correctamente sin necesidad de realizar ningún ajuste.


### 5.3. Ajuste de Columnas INTEGER

**ANÁLISIS DE RESULTADOS**: 

Durante la verificación se analizaron las siguientes columnas de tipo INTEGER:
- `productos.stock`
- `detalle_ordenes.cantidad`
- `carrito.cantidad`
- `resenas_productos.calificacion`

**CONCLUSIÓN**: **No se encontraron tipos por cambiar**. Todas las columnas de tipo INTEGER ya están correctamente configuradas en PostgreSQL. Los datos se están almacenando y recuperando correctamente como enteros sin necesidad de realizar ningún ajuste.


### 5.4. Validación de Columnas ENUM

**ANÁLISIS DE RESULTADOS**: 

Durante la verificación se encontró una diferencia en la representación entre el tipo esperado `ENUM` y el tipo actual `USER-DEFINED (estado_orden)` o `USER-DEFINED (estado_pago)` en las siguientes columnas:
- `ordenes.estado` → Tipo actual: `USER-DEFINED (estado_orden)`
- `historial_pagos.estado_pago` → Tipo actual: `USER-DEFINED (estado_pago)`

**EXPLICACIÓN TÉCNICA**: 

En PostgreSQL, los tipos ENUM se almacenan internamente como tipos `USER-DEFINED` en el sistema de catálogo. Cuando se consulta `information_schema.columns`, el `data_type` aparece como `'USER-DEFINED'` y el `udt_name` contiene el nombre del tipo enum (por ejemplo, `estado_orden`, `estado_pago`).

El código de comparación intenta detectar ENUMs verificando si la palabra "enum" está presente en el `udt_name`, pero como los nombres de los tipos enum son `estado_orden` y `estado_pago` (que no contienen "enum"), la comparación no los identifica correctamente y muestra `USER-DEFINED (nombre_enum)` en lugar de simplemente `ENUM`.

**CONCLUSIÓN**: **NO es necesario realizar ningún cambio**. Los ENUMs están correctamente configurados y funcionando como se espera. La diferencia es solo en cómo se representa en la comparación, pero funcionalmente son tipos ENUM válidos que:
- Restringen los valores permitidos a los definidos en el enum
- Garantizan la integridad de los datos
- Funcionan correctamente en todas las operaciones SQL

Los tipos de datos actuales son correctos y adecuados para almacenar valores enumerados con restricciones de dominio.
